In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle

/opt/conda/lib/python3.8/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
ddbthr = 0.64

In [3]:
templates_array = []

years=['2016APV','2016','2017','2018']
for year in years:
    picklename = year+'/templates.pkl'
    templates_array += [pickle.load(open(picklename,'rb')).sum('msd1','pt1').integrate('systematic','nominal').integrate('mjj',overflow='allnan')]

In [4]:
templates_array

[<Hist (process,region,genflavor,ddb1) instance at 0x7efdebabfe50>,
 <Hist (process,region,genflavor,ddb1) instance at 0x7efdebacbac0>,
 <Hist (process,region,genflavor,ddb1) instance at 0x7efdebacb670>,
 <Hist (process,region,genflavor,ddb1) instance at 0x7efdeba08b50>]

In [5]:
templates = np.sum(templates_array)
templates

<Hist (process,region,genflavor,ddb1) instance at 0x7efe685165b0>

In [6]:
templates.identifiers('process')

[<StringBin (EWKW) instance at 0x7efdebabf040>,
 <StringBin (EWKZ) instance at 0x7efe68510e20>,
 <StringBin (QCD) instance at 0x7efe6853e1f0>,
 <StringBin (VBF) instance at 0x7efdebabf160>,
 <StringBin (VBFDipoleRecoilOff) instance at 0x7efdebabf100>,
 <StringBin (VV) instance at 0x7efe68510ee0>,
 <StringBin (WH) instance at 0x7efdebabf1c0>,
 <StringBin (Wjets) instance at 0x7efe6853e9d0>,
 <StringBin (ZH) instance at 0x7efdebabf220>,
 <StringBin (Zjets) instance at 0x7efe6853e5b0>,
 <StringBin (ZjetsHT) instance at 0x7efe68510ca0>,
 <StringBin (data) instance at 0x7efdebabf2e0>,
 <StringBin (ggF) instance at 0x7efdebabf0a0>,
 <StringBin (muondata) instance at 0x7efdebabf340>,
 <StringBin (singlet) instance at 0x7efe6853e4c0>,
 <StringBin (ttH) instance at 0x7efdebabf280>,
 <StringBin (ttbar) instance at 0x7efe6853e3a0>]

In [7]:
mc = {'QCD':'QCD','Wjets':'Wjets','Zjets':'Zjets','Zjetsbb':'Zjetsbb','EWKV':['EWKW','EWKZ'],'ttbar':'ttbar','singlet':'singlet','VV':'VV','ggF':'ggF','VBF':'VBF','VH':['WH','ZH'],'ttH':'ttH'}
#mc = {'ggF':'ggF','VBF':'VBF','VH':['WH','ZH'],'ttH':'ttH'}
cutflow = pd.DataFrame(columns=['ggffail','ggfpass','vbffail','vbfpass'],index=mc)

In [8]:
templates.integrate('process','QCD').integrate('region',['signal-ggf','signal-vbf']).sum('genflavor','ddb1').values()

{(): 12422439.831586033}

In [9]:
for k,p in mc.items():
    print(p)
    if "Zjets" in p:
        continue
    
    cutflow['ggffail'][k] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['ggfpass'][k] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['vbffail'][k] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    cutflow['vbfpass'][k] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    
cutflow['ggffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]

cutflow['ggffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]



QCD
Wjets
Zjets
Zjetsbb
['EWKW', 'EWKZ']
ttbar
singlet
VV
ggF
VBF
['WH', 'ZH']
ttH


In [10]:
cutflow.astype('int')

,ggffail,ggfpass,vbffail,vbfpass
QCD,11842721,35516,541458,2743
Wjets,310939,1002,8065,25
Zjets,108133,656,2816,17
Zjetsbb,13890,4597,325,116
EWKV,17482,276,2904,64
ttbar,42416,493,1327,16
singlet,5312,53,156,0
VV,4417,97,130,3
ggF,259,103,11,4
VBF,59,27,38,19


In [11]:
(cutflow['ggffail']['ggF']+cutflow['ggfpass']['ggF']+cutflow['ggffail']['VBF']+cutflow['ggfpass']['VBF'])/sum(cutflow['ggffail']+cutflow['ggfpass'])

3.6331004224550446e-05

In [12]:
(cutflow['vbffail']['ggF']+cutflow['vbfpass']['ggF']+cutflow['vbffail']['VBF']+cutflow['vbfpass']['VBF'])/sum(cutflow['vbffail']+cutflow['vbfpass'])

0.0001329328430978816

In [13]:
(cutflow['vbffail']['VBF']+cutflow['vbfpass']['VBF'])/sum(cutflow['vbffail']+cutflow['vbfpass'])

0.00010307196253685578

In [ ]:
cutflow.to_csv('cutflow.csv')